## SENTIMENT ANALYSIS

In [ ]:
import pandas as pd
from textblob import TextBlob


In [ ]:
# Load the data
preprocessed_twitter_tweets = pd.read_csv("preprocessed_twitter_tweets.csv")
preprocessed_reddit_posts = pd.read_csv("preprocessed_reddit_posts.csv")

In [ ]:
# combine the text column for both datasets
merged_df1 = pd.merge(preprocessed_reddit_posts, preprocessed_twitter_tweets, suffixes=('reddit', 'twitter'))
merged_df1['combined_text'] = merged_df1['title'] + ' ' + merged_df1['body'] + ' ' + merged_df1['tweet_text']

In [ ]:
# Add sentiment scores to the dataframe
merged_df1['sentiment'] = merged_df1['combined_text'].apply(lambda text: TextBlob(text).sentiment.polarity)

In [ ]:
# Classify sentiment as positive, negative or neutral
merged_df1['sentiment_class'] = merged_df1['sentiment'].apply(lambda score: 'positive' if score > 0 else 'negative' if score < 0 else 'neutral')


# convert the created_utc column to datetime and extract the date
preprocessed_reddit_posts['date'] = pd.to_datetime(preprocessed_reddit_posts['created_utc']).dt.date
preprocessed_twitter_tweets['date'] = pd.to_datetime(preprocessed_twitter_tweets['created_at']).dt.date

# merge the two datasets on the date column
merged_df1 = pd.merge(preprocessed_reddit_posts, preprocessed_twitter_tweets, on='date', suffixes=('_reddit', '_twitter'))

# combine the text columns from both datasets
merged_df1['merged_df'] = merged_df1['title'] + ' ' + merged_df1['body'] + ' '+ merged_df1['tweet_text']

# Add sentiment scores to the dataframe
merged_df1['sentiment'] = merged_df1['merged_df'].apply(lambda text: TextBlob(text).sentiment.polarity)

# Print the first 5 rows of the dataframe with sentiment scores
merged_df1.head()

In [ ]:
# Classify sentiment as positive, negative or neutral
merged_df1['sentiment_class'] = merged_df1['sentiment'].apply(lambda score: 'positive' if score > 0 else 'negative' if score < 0 else 'neutral')

# Calculate percentage of positive, negative, and neutral sentiments
num_positive = len(merged_df1[merged_df1['sentiment_class'] == 'positive'])
num_negative = len(merged_df1[merged_df1['sentiment_class'] == 'negative'])
num_neutral = len(merged_df1[merged_df1['sentiment_class'] == 'neutral'])

total = num_positive + num_negative + num_neutral

percent_positive = (num_positive / total) * 100
percent_negative = (num_negative / total) * 100
percent_neutral = (num_neutral / total) * 100

print(f"Percentage of positive sentiments: {percent_positive:.2f}%")
print(f"Percentage of negative sentiments: {percent_negative:.2f}%")
print(f"Percentage of neutral sentiments: {percent_neutral:.2f}%")


In [ ]:
# save the merged dataset to a new file
merged_df1.to_csv('sentiment_analysis.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

# Count the number of observations in each sentiment class
sentiment_counts = merged_df1['sentiment_class'].value_counts()

# Plot a bar chart of the sentiment counts
plt.bar(sentiment_counts.index, sentiment_counts.values)
plt.title('Sentiment distribution')
plt.xlabel('Sentiment class')
plt.ylabel('Number of observations')
plt.show()


In [ ]:
# Plot a histogram of the sentiment scores
plt.hist(merged_df1['sentiment'], bins=20)
plt.title('Sentiment score distribution')
plt.xlabel('Sentiment score')
plt.ylabel('Number of observations')
plt.show()


In [ ]:
import seaborn as sns
# Create a new column with the length of the combined text
merged_df1['text_length'] = merged_df1['combined_text'].apply(len)

# Plot a scatter plot of sentiment score vs text length
sns.scatterplot(x='text_length', y='sentiment', data=merged_df1)
plt.title('Sentiment score vs text length')
plt.xlabel('Text length')
plt.ylabel('Sentiment score')
plt.show()


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from palettable.colorbrewer.qualitative import Dark2_8

# Combine all the text with positive sentiment into a single string
positive_text = ' '.join(merged_df1.loc[merged_df1['sentiment_class'] == 'positive', 'combined_text'])
positive_text = positive_text.replace("'", "") # remove apostrophes

# Generate the positive sentiment wordcloud
positive_wordcloud = WordCloud(width=1200, height=800, background_color='white', colormap=Dark2_8.mpl_colormap).generate(positive_text)

# Combine all the text with negative sentiment into a single string
negative_text = ' '.join(merged_df1.loc[merged_df1['sentiment_class'] == 'negative', 'combined_text'])
negative_text = negative_text.replace("'", "") # remove apostrophes

# Generate the negative sentiment wordcloud
negative_wordcloud = WordCloud(width=1200, height=800, background_color='white', colormap=Dark2_8.mpl_colormap).generate(negative_text)


In [ ]:
# Display the positive sentiment wordcloud
plt.figure(figsize=(8, 8), facecolor=None)
plt.imshow(positive_wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()


In [ ]:
# Display the negative sentiment wordcloud
plt.figure(figsize=(8, 8), facecolor=None)
plt.imshow(negative_wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()